<a href="https://colab.research.google.com/github/AnaCam2601/Carga-Datos-Inventario/blob/main/CARGA_AUTOMATICA_DE_DATOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga los datos del sheets original para pegarlos en el sheets de busqueda rapida


In [ ]:
# ✅ Instala las librerías necesarias para Google Sheets y manejo de Excel
!pip install gspread pandas openpyxl gdown schedule


In [ ]:
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
from datetime import datetime
import time
import schedule

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


In [ ]:
CONFIG = {
    "MEDICAMENTOS HOSPITALARIA": {
        "rango_origen": "A1:BV631",
        "columnas_seleccionadas": [0, 1, 2, 3, 4, 5, 6, 71],
        "columnas_origen": ["147", "CLAVE", "DESCRIPCION", "CADUCIDAD", "SEMAFORO", "UNIDAD",
                            "SALDO INICIAL MARZO 2025", "SALDO FINAL MARZO 2025"],
        "rango_destino": "A1:H631"
    },
    "MATERIAL DE CURACION HOSPITALAR": {
        "rango_origen": "A1:BV589",
        "columnas_seleccionadas": [0, 1, 2, 3, 4, 5, 6, 71],
        "columnas_origen": ["NO.", "CLAVE", "DESCRIPCION", "CADUCIDAD", "SEMAFORO", "UNIDAD",
                            "SALDO INICIAL MARZO 2025", "SALDO FINAL MARZO 2025"],
        "rango_destino": "K1:R589"
    },
    "REACTIVOS HOSPITALARIA": {
        "rango_origen": "A2:BV70",
        "columnas_seleccionadas": [0, 1, 2, 3, 4, 5, 6, 71],
        "columnas_origen": ["NO.", "CLAVE", "DESCRIPCION", "CADUCIDAD", "SEMAFORO",
                            "UNIDAD", "SALDO INICIAL MARZO 2025", "SALDO FINAL MARZO 2025"],
        "rango_destino": "U1:AB70"
    },
    "MEDICAMENTOS GRATUITA": {
        "rango_origen": "A1:BW383",
        "columnas_seleccionadas": [0, 1, 2, 3, 4, 5, 6, 7, 72],
        "columnas_origen": ["NO.", "CLAVE", "DESCRIPCION", "CADUCIDAD", "LOTE", "SEMAFORO",
                            "UNIDAD", "SALDO INICIAL MARZO 2025", "SALDO FINAL MARZO 2025"],
        "rango_destino": "AE1:AM383"
    },
    "MATERIAL DE CURACIÓN GRATUITA": {
        "rango_origen": "A1:BW100",
        "columnas_seleccionadas": [0, 1, 2, 3, 4, 5, 6, 7, 72],
        "columnas_origen": ["NO.", "CLAVE", "DESCRIPCION", "CADUCIDAD", "LOTE", "SEMAFORO",
                            "UNIDAD", "SALDO INICIAL MARZO 2025", "SALDO FINAL MARZO 2025"],
        "rango_destino": "AO1:AW100"
    }
}


In [ ]:
def limpiar_dato(valor):
    if pd.isna(valor) or valor == "":
        return ""
    if isinstance(valor, datetime):
        return valor.strftime('%d/%m/%Y')
    if isinstance(valor, (int, float)):
        return str(int(valor))
    return str(valor).strip()


In [ ]:
print("Cargando compendio de claves...")
compendio_url = "https://docs.google.com/spreadsheets/d/12B9trRSZVzH3TI0cSrOx_87IG5ip2r1BhpJrXnci9YI/export?format=xlsx"
compendio_df = pd.read_excel(compendio_url, sheet_name="CNIS", usecols=[0, 1])

compendio_df.columns = ["Clave", "Descripcion"]
compendio_df["Clave"] = compendio_df["Clave"].astype(str).str.strip()
compendio_df["Descripcion"] = compendio_df["Descripcion"].astype(str).str.strip()

diccionario_descripciones = dict(zip(compendio_df["Clave"], compendio_df["Descripcion"]))


Cargando compendio de claves...


In [ ]:
def cargar_datos_seguro():
    try:
        print("Descarga desde el archivo original...")
        !gdown --id "1t4yBRdnf2z4z33ijB9QgsG2vOYrMAYhh" -O temp_inventario.xlsx -q 2>/dev/null

        print("Conexión con la hoja de Búsqueda Rápida...")
        sheet = gc.open_by_key("12ULrg_sUNmXDgMy8ydQkRoAbrJ1V1ckEL4sw-H7EfvM").worksheet("DATOSREAL")

        for hoja, config in CONFIG.items():
            try:
                print(f"\nProcesando {hoja}...")

                def get_nrows(rango):
                    parts = rango.split(":")
                    if len(parts) == 2:
                        return int(''.join(filter(str.isdigit, parts[1])))
                    return 631

                df = pd.read_excel(
                    "temp_inventario.xlsx",
                    sheet_name=hoja,
                    usecols=config["columnas_seleccionadas"],
                    nrows=get_nrows(config["rango_origen"])
                )
                df.columns = config["columnas_origen"]
                df = df.applymap(limpiar_dato)
                df = df.dropna(how='all')

                # Reemplazo de descripción según clave
                if "CLAVE" in df.columns and "DESCRIPCION" in df.columns:
                    df["DESCRIPCION"] = df.apply(
                        lambda row: diccionario_descripciones.get(row["CLAVE"], row["DESCRIPCION"]),
                        axis=1
                    )

                datos = df.values.tolist()
                batch_size = 100

                celda_inicial, _ = config["rango_destino"].split(":")
                fila_base, col_base = gspread.utils.a1_to_rowcol(celda_inicial)

                for i in range(0, len(datos), batch_size):
                    batch = datos[i:i + batch_size]
                    fila_inicio = fila_base + i
                    fila_fin = fila_inicio + len(batch) - 1
                    col_inicio = col_base
                    col_fin = col_inicio + len(batch[0]) - 1 if batch else col_base

                    rango = f"{gspread.utils.rowcol_to_a1(fila_inicio, col_inicio)}:" \
                            f"{gspread.utils.rowcol_to_a1(fila_fin, col_fin)}"
                    print(f"   Actualizando {rango}...")

                    sheet.update(
                        values=batch,
                        range_name=rango
                    )

                print(f"{hoja} cargado correctamente - {len(datos)} filas")

            except Exception as e:
                print(f"Error en {hoja}: {str(e)}")
                continue

    except Exception as e:
        print(f"Error crítico: {str(e)}")
    finally:
        print("\nLimpieza del archivo temporal...")
        !rm -f temp_inventario.xlsx


In [ ]:
def ejecutar_carga():
    print("🔥 Webhook activado - Iniciando actualización...")
    print("\n" + "="*60)
    print(f"INICIO DE CARGA - {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
    inicio = time.time()

    cargar_datos_seguro()

    print(f"\nCARGA COMPLETADA EN {time.time() - inicio:.2f} segundos")
    print("="*60 + "\n")

# Ejecutar de inmediato
ejecutar_carga()

# Programar para cada 24 horas
schedule.every(24).hours.do(ejecutar_carga)


🔥 Webhook activado - Iniciando actualización...

INICIO DE CARGA - 08/04/2025 22:21:10
Descarga desde el archivo original...
Conexión con la hoja de Búsqueda Rápida...

Procesando MEDICAMENTOS HOSPITALARIA...


<ipython-input-42-10ddf5f49836>:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_dato)


   Actualizando A1:H100...
   Actualizando A101:H200...
   Actualizando A201:H300...
   Actualizando A301:H400...
   Actualizando A401:H500...
   Actualizando A501:H600...
   Actualizando A601:H631...
MEDICAMENTOS HOSPITALARIA cargado correctamente - 631 filas

Procesando MATERIAL DE CURACION HOSPITALAR...


<ipython-input-42-10ddf5f49836>:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_dato)


   Actualizando K1:R100...
   Actualizando K101:R200...
   Actualizando K201:R300...
   Actualizando K301:R400...
   Actualizando K401:R500...
   Actualizando K501:R589...
MATERIAL DE CURACION HOSPITALAR cargado correctamente - 589 filas

Procesando REACTIVOS HOSPITALARIA...


...

<ipython-input-42-10ddf5f49836>:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_dato)


   Actualizando AE1:AM100...
   Actualizando AE101:AM200...
   Actualizando AE201:AM300...
   Actualizando AE301:AM383...
MEDICAMENTOS GRATUITA cargado correctamente - 383 filas

Procesando MATERIAL DE CURACIÓN GRATUITA...


<ipython-input-42-10ddf5f49836>:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_dato)


   Actualizando AO1:AW94...
MATERIAL DE CURACIÓN GRATUITA cargado correctamente - 94 filas

Limpieza del archivo temporal...

CARGA COMPLETADA EN 15.98 segundos



Every 24 hours do ejecutar_carga() (last run: [never], next run: 2025-04-09 22:21:26)